In [6]:
## Run selenium and chrome driver to scrape data from cloudbytes.dev
import time
import os.path
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [7]:
## Setup chrome options
chrome_options = Options()
chrome_options.headless = True # Ensure GUI is off
# chrome_options.add_argument("--window-size=1920,1200")

In [8]:
# Set path to chromedriver as per your configuration
homedir = os.path.expanduser("~")
webdriver_service = Service(f"{homedir}/ao3lockwood-co/chromedriver")

In [9]:
# Choose Chrome Browser
browser = webdriver.Chrome(service=webdriver_service, options=chrome_options)

In [10]:
# Get page
pagenum=1
link="https://archiveofourown.org/tags/Lockwood%20*a*%20Co*d*%20-%20Jonathan%20Stroud/works?page="+str(pagenum)
#link="https://archiveofourown.org/tags/Lockwood%20*a*%20Co*d*%20-%20Jonathan%20Stroud/works?page=1"
browser.get(link)

maxpagenum=int(browser.find_element(By.XPATH,'//ol[1]/li[13]').text.strip())

In [11]:
def get_links(browser):
    works = browser.find_elements(By.XPATH, '//ol[2]/li')
    # Iterate through each work and extract author and datetime
    data=[]
    for work in works:
        h4 = work.find_element(By.TAG_NAME,'h4')
        a = h4.find_elements(By.TAG_NAME, 'a')
        links = a[0].get_attribute("href")
        data.append(links)
    return data
    

In [12]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
 
print("now =", now)

# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)

# Create an empty DataFrame to hold the data
data_list = get_links(browser)
print('page 1 has been processed')
# Iterate through each page and append the data to the DataFrame
for p in range(2,maxpagenum+1):
    pagenum=p
    time.sleep(10)
    print(f'procesing page {pagenum}/{maxpagenum}')
    link="https://archiveofourown.org/tags/Lockwood%20*a*%20Co*d*%20-%20Jonathan%20Stroud/works?page="+str(pagenum)
    browser.get(link)
    data_list=data_list + (get_links(browser))
    print(len(data_list))

#Wait for 10 seconds
time.sleep(10)
browser.quit()

now = 2023-04-25 10:46:00.113417
date and time = 25/04/2023 10:46:00
page 1 has been processed
procesing page 2/68
40
procesing page 3/68
60
procesing page 4/68
80
procesing page 5/68
100
procesing page 6/68
120
procesing page 7/68
140
procesing page 8/68
160
procesing page 9/68
180
procesing page 10/68
200
procesing page 11/68
220
procesing page 12/68
240
procesing page 13/68
260
procesing page 14/68
280
procesing page 15/68
300
procesing page 16/68
320
procesing page 17/68
340
procesing page 18/68
360
procesing page 19/68
380
procesing page 20/68
400
procesing page 21/68
420
procesing page 22/68
440
procesing page 23/68
460
procesing page 24/68
480
procesing page 25/68
500
procesing page 26/68
520
procesing page 27/68
540
procesing page 28/68
560
procesing page 29/68
580
procesing page 30/68
600
procesing page 31/68
620
procesing page 32/68
640
procesing page 33/68
660
procesing page 34/68
680
procesing page 35/68
700
procesing page 36/68
720
procesing page 37/68
740
procesing page 3

In [13]:
data_list

['https://archiveofourown.org/works/46183210',
 'https://archiveofourown.org/works/45088573',
 'https://archiveofourown.org/works/46429513',
 'https://archiveofourown.org/works/46673722',
 'https://archiveofourown.org/works/46729345',
 'https://archiveofourown.org/works/45773677',
 'https://archiveofourown.org/works/46729273',
 'https://archiveofourown.org/works/45376639',
 'https://archiveofourown.org/works/46315813',
 'https://archiveofourown.org/works/46266553',
 'https://archiveofourown.org/works/46724731',
 'https://archiveofourown.org/works/46724329',
 'https://archiveofourown.org/works/46721113',
 'https://archiveofourown.org/works/46719808',
 'https://archiveofourown.org/works/46234456',
 'https://archiveofourown.org/works/46616701',
 'https://archiveofourown.org/works/45196636',
 'https://archiveofourown.org/works/46481659',
 'https://archiveofourown.org/works/46709671',
 'https://archiveofourown.org/works/46545298',
 'https://archiveofourown.org/works/46452142',
 'https://arc

In [14]:
def get_data(links_list):
    data=[]
    counter=0
    for x in links_list:
        newlink=x+'?view_adult=true'
        counter+=1
        print(f"getting missing data {counter}/{len(links_list)}")
        source = requests.get(newlink, headers={
                          'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64)'}).text
        soup = BeautifulSoup(source,'html.parser')
        title=soup.find('h2', attrs={'class':'title heading'}).get_text().replace('\n','').strip()
        try:
            author=soup.find('a', attrs={'rel':'author'}).get_text()
        except:
            author="Anonymous"
        datetime=soup.find('dd', attrs={'class':'published'}).get_text()
        try:
            updated = soup.find('dd', attrs={'class':'status'}).get_text()
        except:
            updated=datetime
        chapters=soup.find('dd', attrs={'class':'chapters'}).get_text()
        language=soup.find('dd', attrs={'class':'language'}).get_text().replace('\n','').strip()
        words=soup.find('dd', attrs={'class':'words'}).get_text()
        try:
            kudos=soup.find('dd', attrs={'class':'kudos'}).get_text()
        except:
            kudos=0
        try:
            comments=soup.find('dd', attrs={'class':'comments'}).get_text()
        except:
            comments=0
        try:
            bookmarks=soup.find('dd', attrs={'class':'bookmarks'}).get_text()
        except:
            bookmarks=0
        try:
            hits=soup.find('dd', attrs={'class':'hits'}).get_text()
        except:
            hits=0
        warning=soup.find('dd', attrs={'class':'warning tags'}).get_text().replace('\n','').strip()
        try:
            ships = soup.find('dd', attrs={'class':'relationship tags'})
            ships_list=ships.find_all('a', attrs={'class':'tag'})
            mainrelationship = ships_list[0].get_text().strip()
            relationship_list = []
            for r in ships_list:
                relationship_list.append(r.get_text().strip())
        except:
            mainrelationship='None'
            relationship_list = []
        try:
            char = soup.find('dd', attrs={'class':'character tags'})
            char_list=char.find_all('a', attrs={'class':'tag'})
            character_list = []
            for c in char_list:
                character_list.append(c.get_text().strip())
        except:
            character_list=[]
        try:
            freeform = soup.find('dd', attrs={'class':'freeform tags'})
            freefom_list=freeform.find_all('a', attrs={'class':'tag'})
            tags_list = []
            for t in freefom_list:
                tags_list.append(t.get_text().strip())
        except:
            tags_list = []
        try:
            position = soup.find('span', attrs={'class':'position'})
            position_list = position.find('a')
            series_list = []
            for p in position_list:
                series_list.append(p.get_text().strip())
        except:
            series_list = 'not a series'
        try:
            summary=soup.find('div', attrs={'class':'summary module'}).get_text().replace('\n', ' ').replace('Summary:','').strip()
        except:
            summary=np.nan
        rating_tag=soup.find('dd', attrs={'class':'rating tags'}).get_text().replace('\n','').strip()
        row = {'link':x, 
               'title':title,
               'author':author,
               'published':datetime, 
               'updatedate':updated,
               'chapters':chapters,
               'language':language,
               'words':words,
               'kudos':kudos, 
               'comments':comments,
               'bookmarks':bookmarks,
               'hits':hits,
               'warning':warning,
               'mainship':mainrelationship,
               'relationship':relationship_list,
               'characters':character_list, 
               'tags':tags_list,
               'summary':summary,
               'rating':rating_tag
               }
        #print(row)
        data.append(row)
        time.sleep(10)
    return pd.DataFrame(data)

In [15]:
final=get_data(data_list)
# Split the chapter column into chapter and chapter_max, and create a completion column
final[['chapter','chapter_max']] = final.chapters.str.split("/", expand=True)
final['completion'] = final.apply(lambda row: 'completed' if row['chapter']==row['chapter_max'] else 'incomplete', axis=1)


getting missing data 1/1347
getting missing data 2/1347
getting missing data 3/1347
getting missing data 4/1347
getting missing data 5/1347
getting missing data 6/1347
getting missing data 7/1347
getting missing data 8/1347
getting missing data 9/1347
getting missing data 10/1347
getting missing data 11/1347
getting missing data 12/1347
getting missing data 13/1347
getting missing data 14/1347
getting missing data 15/1347
getting missing data 16/1347
getting missing data 17/1347


KeyboardInterrupt: 

In [ ]:
final.to_csv('ao3_lockwood_and_co_ao_21042023_1857.csv', index=False)